## Train a Single agent system using RLLib
This notebook will demonstrate how to use PrimaiteRayEnv to train a basic PPO agent.

In [ ]:
from primaite.game.game import PrimaiteGame
import yaml
from primaite.config.load import data_manipulation_config_path

from primaite.session.environment import PrimaiteRayEnv
from ray.rllib.algorithms import ppo
from ray import air, tune
import ray
from ray.rllib.algorithms.ppo import PPOConfig

# If you get an error saying this config file doesn't exist, you may need to run `primaite setup` in your command line
# to copy the files to your user data path.
with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)

ray.init(local_mode=True)


#### Create a Ray algorithm and pass it our config.

In [ ]:
for agent in cfg['agents']:
    if agent["ref"] == "defender":
        agent['agent_settings']['flatten_obs'] = True
env_config = cfg

config = (
    PPOConfig()
    .environment(env=PrimaiteRayEnv, env_config=env_config, disable_env_checking=True)
    .rollouts(num_rollout_workers=0)
    .training(train_batch_size=128)
)


#### Set training parameters and start the training

In [ ]:
tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"timesteps_total": 5 * 128}
    ),
    param_space=config
).fit()
